<a href="https://colab.research.google.com/github/vesyapdw/Komputasi-Intelejensia-Tasks/blob/main/TaskWeek6_Vesya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama: Vesya Padmadewi

NPM: 2106726926

Original Code: https://www.kaggle.com/code/yasmeensharaan/twitter-sentiment-using-lstm

In [ ]:
import numpy as np
import pandas as pd
import re    #Regular Expressions ( re ) module in Python
# Instead of keras.preprocessing.text, use tensorflow.keras.preprocessing.text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences # Use tensorflow.keras.utils
from tensorflow.keras.models import Sequential # Use tensorflow.keras.models
from tensorflow.keras.layers import Dense, Embedding, LSTM # Use tensorflow.keras.layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping # Use tensorflow.keras.callbacks

In [ ]:
data = pd.read_csv('/content/twitter_training.csv',header=None)
data.head()

,0,1,2,3
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


# Preprocessing

In [ ]:
new_columns=["id","type","sentiment","text"]
data.columns = new_columns
data.head()

,id,type,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         74682 non-null  int64 
 1   type       74682 non-null  object
 2   sentiment  74682 non-null  object
 3   text       73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [ ]:
# Drop columns that isn't important
data=data[['text','sentiment']]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       73996 non-null  object
 1   sentiment  74682 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
Negative,22542
Positive,20832
Neutral,18318
Irrelevant,12990


In [ ]:
#select positive & negative only
data = data[data.sentiment != "Neutral"]
data = data[data.sentiment != "Irrelevant"]

data

,text,sentiment
0,im getting on borderlands and i will murder yo...,Positive
1,I am coming to the borders and I will kill you...,Positive
2,im getting on borderlands and i will kill you ...,Positive
3,im coming on borderlands and i will murder you...,Positive
4,im getting on borderlands 2 and i will murder ...,Positive
...,...,...
74677,Just realized that the Windows partition of my...,Positive
74678,Just realized that my Mac window partition is ...,Positive
74679,Just realized the windows partition of my Mac ...,Positive
74680,Just realized between the windows partition of...,Positive


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
Negative,22542
Positive,20832


In [ ]:
data['sentiment'] = data['sentiment'].astype(str)

In [ ]:
data['text'] = data['text'].astype(str)

In [ ]:
data['text'] = data['text'].apply(lambda x: x.lower())
data

,text,sentiment
0,im getting on borderlands and i will murder yo...,Positive
1,i am coming to the borders and i will kill you...,Positive
2,im getting on borderlands and i will kill you ...,Positive
3,im coming on borderlands and i will murder you...,Positive
4,im getting on borderlands 2 and i will murder ...,Positive
...,...,...
74677,just realized that the windows partition of my...,Positive
74678,just realized that my mac window partition is ...,Positive
74679,just realized the windows partition of my mac ...,Positive
74680,just realized between the windows partition of...,Positive


In [ ]:
data['text'] = data['text'].apply((lambda x: re.sub('[^a-z0-9\s]','',x))) #The syntax for re. sub() is re. sub(pattern,repl,string).
data

,text,sentiment
0,im getting on borderlands and i will murder yo...,Positive
1,i am coming to the borders and i will kill you...,Positive
2,im getting on borderlands and i will kill you all,Positive
3,im coming on borderlands and i will murder you...,Positive
4,im getting on borderlands 2 and i will murder ...,Positive
...,...,...
74677,just realized that the windows partition of my...,Positive
74678,just realized that my mac window partition is ...,Positive
74679,just realized the windows partition of my mac ...,Positive
74680,just realized between the windows partition of...,Positive


# Build the Tokenizer

In [ ]:
top_words = 10000
tokenizer = Tokenizer(num_words=top_words, split=' ')
tokenizer.fit_on_texts(data['text'].values)

In [ ]:
word_index =  tokenizer.word_index
#word_index

In [ ]:
len(word_index)

23315

In [ ]:
X = tokenizer.texts_to_sequences(data['text'].values)
print(len(X))
print(X[0])


43374
[29, 130, 14, 106, 4, 2, 58, 1448, 13, 26]


In [ ]:
# padding
X = pad_sequences(X)
print(X[0])


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0   29  130   14  106    4    2   58 1448   13   26]


In [ ]:
len(X[0])

166

# Build the model

In [ ]:
embed_dim = 128
lstm_units = 196

model = Sequential()
model.add(Embedding(top_words, embed_dim,input_length = X.shape[1]))
#model.add(LSTM(lstm_units))

model.add(LSTM(100,dropout=0.25,recurrent_dropout=0.25))
model.add(Dense(50,activation='relu'))
model.add(Dense(25,activation='relu'))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
parameters_of_Embedding_layer=top_words*embed_dim
parameters_of_Embedding_layer

1280000

In [ ]:
parameters_of_lstm_layer=(embed_dim+lstm_units+1)*lstm_units*4
parameters_of_lstm_layer

254800

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y=le.fit_transform(data['sentiment'])
print(le.classes_)
print(y)

['Negative' 'Positive']
[1 1 1 ... 1 1 1]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(29060, 166) (29060,)
(14314, 166) (14314,)


In [ ]:
batch_size = 32
model.fit(X_train, y_train, epochs = 7, batch_size=batch_size,validation_data= (X_test, y_test)
                             ,callbacks=EarlyStopping(patience=3,restore_best_weights=True))

Epoch 1/7
909/909 ━━━━━━━━━━━━━━━━━━━━ 303s 325ms/step - accuracy: 0.7332 - loss: 0.5020 - val_accuracy: 0.8830 - val_loss: 0.2758
Epoch 2/7
909/909 ━━━━━━━━━━━━━━━━━━━━ 307s 314ms/step - accuracy: 0.9091 - loss: 0.2156 - val_accuracy: 0.8910 - val_loss: 0.2527
Epoch 3/7
909/909 ━━━━━━━━━━━━━━━━━━━━ 322s 314ms/step - accuracy: 0.9365 - loss: 0.1494 - val_accuracy: 0.9194 - val_loss: 0.2001
Epoch 4/7
909/909 ━━━━━━━━━━━━━━━━━━━━ 286s 315ms/step - accuracy: 0.9473 - loss: 0.1140 - val_accuracy: 0.9187 - val_loss: 0.2141
Epoch 5/7
909/909 ━━━━━━━━━━━━━━━━━━━━ 322s 315ms/step - accuracy: 0.9563 - loss: 0.0928 - val_accuracy: 0.9251 - val_loss: 0.2040
Epoch 6/7
909/909 ━━━━━━━━━━━━━━━━━━━━ 323s 316ms/step - accuracy: 0.9624 - loss: 0.0800 - val_accuracy: 0.9240 - val_loss: 0.2134


In [ ]:
model.evaluate(X_train, y_train)

909/909 ━━━━━━━━━━━━━━━━━━━━ 66s 73ms/step - accuracy: 0.9572 - loss: 0.0964


[0.09925667196512222, 0.956434965133667]

In [ ]:
model.evaluate(X_test, y_test)

448/448 ━━━━━━━━━━━━━━━━━━━━ 31s 70ms/step - accuracy: 0.9209 - loss: 0.2052


[0.20013532042503357, 0.9193796515464783]

In [ ]:
model.save('twitter_sentiment.h5')

# Prediction

In [ ]:
import numpy as np
y_pred=(model.predict(X_test)>=0.5).astype(int)
y_pred

448/448 ━━━━━━━━━━━━━━━━━━━━ 32s 71ms/step


array([[1],
       [1],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [ ]:
data['text'].values[3000]

'i only know that i will be wearing one from now on along with gloves its going to be my new norm most of the public still cannot find masks if u order off of amazon it can take up to a month to get any type of mask anyone else having this problem  '

In [ ]:
data['sentiment'].values[3000]

'Negative'

In [ ]:
twt = ['i only know that i will be wearing one from now on along with gloves its going to be my new norm most of the public still cannot find masks if u order off of amazon it can take up to a month to get any type of mask anyone else having this problem  ']

twt = tokenizer.texts_to_sequences(twt)

twt


[[2,
  96,
  112,
  15,
  2,
  58,
  25,
  1675,
  47,
  35,
  34,
  14,
  1058,
  18,
  5359,
  28,
  87,
  3,
  25,
  12,
  54,
  8330,
  129,
  7,
  1,
  1532,
  63,
  544,
  364,
  1274,
  55,
  190,
  476,
  160,
  7,
  157,
  9,
  38,
  253,
  61,
  3,
  5,
  361,
  3,
  32,
  170,
  1484,
  7,
  1093,
  308,
  445,
  232,
  8,
  268]]

In [ ]:
twt = pad_sequences(twt, maxlen=28)
print(twt)

[[  63  544  364 1274   55  190  476  160    7  157    9   38  253   61
     3    5  361    3   32  170 1484    7 1093  308  445  232    8  268]]


In [ ]:
print(model.predict(twt))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step
[[0.01318181]]


In [ ]:
prediction_twt=y_pred=(model.predict(twt)>=0.5).astype(int)
prediction_twt

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


array([[0]])